In [18]:
import pandas as pd
import numpy as np

##  Wczytywanie i łączenie tabeli z danymi dot. FID,IID,SEX,AGE

In [19]:
df_sex_age = pd.read_excel(r"C:\Users\lancernik\Desktop\Hackaton\data\Pheno_SEX_AGE_ZAKODOWANE.xlsx")
df_log_bmi = pd.read_excel(r"C:\Users\lancernik\Desktop\Hackaton\data\Pheno_log_BMI_zakodowany.xlsx")
df_age = df_sex_age.AGE
df_log_bmi.insert(2, "Age", df_age, True)
df_polaczony = pd.merge(df_sex_age,df_log_bmi,on="IID")
df_polaczony = df_polaczony.drop(columns='SEX_y')
df_polaczony['index_bmi'] = df_polaczony['log_BMI'].apply(lambda x:2 if x>= 1.397940 else 1 )

In [20]:
df_polaczony.head(5)

,FID_x,IID,SEX_x,AGE,FID_y,Age,log_BMI,index_bmi
0,3,1,2,64,3,64,1.410102,2
1,4,2,2,29,4,29,1.391112,1
2,5,3,1,64,5,64,1.381837,1
3,6,4,2,21,6,21,1.290702,1
4,7,5,2,38,7,38,1.494572,2


# Dodajemy kolumny z allelami na snp

In [21]:
pedtry = pd.read_table(r"C:\Users\lancernik\Desktop\Hackaton\BioNinjaHack_obesity.ped",sep=" ",header=None)
pedtry.rename(columns={5:'AAlog_BMI',0:'FID', 1:'IID',2:'M',3:'F',4:'Gender'}, 
                 inplace=True)

pedtry['AAlog_BMI'] = df_polaczony['log_BMI']
pedtry['AGE'] = df_polaczony['AGE']

In [22]:
pedtry.head(5)

,FID,IID,M,F,Gender,AAlog_BMI,6,7,8,9,...,655,656,657,658,659,660,661,662,663,AGE
0,3,1,0,0,2,1.410102,G,G,G,A,...,G,T,T,G,G,G,G,G,G,64.0
1,4,2,0,0,2,1.391112,G,G,G,G,...,G,T,T,G,G,G,G,G,G,29.0
2,5,3,0,0,1,1.381837,G,G,A,A,...,G,T,T,G,G,G,G,G,G,64.0
3,6,4,0,0,2,1.290702,A,G,A,A,...,G,T,T,G,G,G,G,G,G,21.0
4,7,5,0,0,2,1.494572,A,G,A,A,...,G,T,T,G,G,G,G,G,G,38.0


# Tworzymy listę par alleli (kol. check) w celu zbadania wystąpienia snp w próbkach

In [23]:
snipy = pd.read_table(r"C:\Users\lancernik\Desktop\Hackaton\BioNinjaHack_obesity.bim",sep="\t",header=None)
snipy['Check'] = snipy[4]+snipy[5]
snipy.drop(columns=[2,3])
snipy.head(5)

,0,1,2,3,4,5,Check
0,2,rs12620338,225.442,223064768,A,G,AG
1,2,rs7559271,225.448,223068286,G,A,GA
2,2,rs2234675,225.480,223085955,T,G,TG
3,2,rs6436302,225.517,223106322,T,C,TC
4,2,rs12053273,225.517,223106526,A,G,AG


In [24]:
snipnazwa = snipy[1].tolist()
print(snipnazwa[:10])

['rs12620338', 'rs7559271', 'rs2234675', 'rs6436302', 'rs12053273', 'rs1430657', 'rs16863576', 'rs7589708', 'rs4674639', 'rs10932949']


# Tworzymy pary alleli w celu badania wystąpienia snp

In [25]:
for i in range(0,658,2):
    pedtry[snipnazwa[int(i/2)]] = pedtry[6+i]+pedtry[7+i]

In [26]:
pedtry.head(5)

,FID,IID,M,F,Gender,AAlog_BMI,6,7,8,9,...,rs837550,rs2287072,rs112426189,rs3744374,rs12602590,rs11654604,rs200805689,rs117651561,rs79742527,rs143040759
0,3,1,0,0,2,1.410102,G,G,G,A,...,AA,AA,TG,AG,CC,GG,TT,GG,GG,GG
1,4,2,0,0,2,1.391112,G,G,G,G,...,GG,AA,GG,GG,CC,AG,TT,GG,GG,GG
2,5,3,0,0,1,1.381837,G,G,A,A,...,GA,AA,GG,AA,CC,GG,TT,GG,GG,GG
3,6,4,0,0,2,1.290702,A,G,A,A,...,GG,AA,GG,GG,CC,GG,TT,GG,GG,GG
4,7,5,0,0,2,1.494572,A,G,A,A,...,GA,AA,TG,AG,CC,GG,TT,GG,GG,GG


In [27]:
colList = ['FID', 'IID','M','F','Gender', 'AAlog_BMI']
colList2 = list(pedtry.columns)
colList2 = colList2[664:]
colList.extend(colList2)
pedtry = pedtry[colList]

In [28]:
pedtry.head(5)

,FID,IID,M,F,Gender,AAlog_BMI,AGE,rs12620338,rs7559271,rs2234675,...,rs837550,rs2287072,rs112426189,rs3744374,rs12602590,rs11654604,rs200805689,rs117651561,rs79742527,rs143040759
0,3,1,0,0,2,1.410102,64.0,GG,GA,GG,...,AA,AA,TG,AG,CC,GG,TT,GG,GG,GG
1,4,2,0,0,2,1.391112,29.0,GG,GG,GG,...,GG,AA,GG,GG,CC,AG,TT,GG,GG,GG
2,5,3,0,0,1,1.381837,64.0,GG,AA,GG,...,GA,AA,GG,AA,CC,GG,TT,GG,GG,GG
3,6,4,0,0,2,1.290702,21.0,AG,AA,GG,...,GG,AA,GG,GG,CC,GG,TT,GG,GG,GG
4,7,5,0,0,2,1.494572,38.0,AG,AA,TG,...,GA,AA,TG,AG,CC,GG,TT,GG,GG,GG


In [29]:
pedtry = pedtry.drop(columns=['M','F'])
pedtry.head(5)

,FID,IID,Gender,AAlog_BMI,AGE,rs12620338,rs7559271,rs2234675,rs6436302,rs12053273,...,rs837550,rs2287072,rs112426189,rs3744374,rs12602590,rs11654604,rs200805689,rs117651561,rs79742527,rs143040759
0,3,1,2,1.410102,64.0,GG,GA,GG,TT,AA,...,AA,AA,TG,AG,CC,GG,TT,GG,GG,GG
1,4,2,2,1.391112,29.0,GG,GG,GG,TC,AG,...,GG,AA,GG,GG,CC,AG,TT,GG,GG,GG
2,5,3,1,1.381837,64.0,GG,AA,GG,CC,GG,...,GA,AA,GG,AA,CC,GG,TT,GG,GG,GG
3,6,4,2,1.290702,21.0,AG,AA,GG,TC,GG,...,GG,AA,GG,GG,CC,GG,TT,GG,GG,GG
4,7,5,2,1.494572,38.0,AG,AA,TG,TC,AG,...,GA,AA,TG,AG,CC,GG,TT,GG,GG,GG


In [30]:
snipy.head()

,0,1,2,3,4,5,Check
0,2,rs12620338,225.442,223064768,A,G,AG
1,2,rs7559271,225.448,223068286,G,A,GA
2,2,rs2234675,225.480,223085955,T,G,TG
3,2,rs6436302,225.517,223106322,T,C,TC
4,2,rs12053273,225.517,223106526,A,G,AG


# Porównujemy klucz z allelami u poszczególnych osób
W przypadku zgodnosci przypisujemy klasę 1, w przypadku braku zgodności przypisujemy klasę 0.

In [33]:
listaSnipow = snipy.Check.tolist()
columns = snipnazwa

In [34]:
a = listaSnipow
lookup = {key : (value,) for key, value in zip(columns, a)}



columns = snipnazwa
pedtry.loc[:, columns] = pedtry.loc[:, columns].isin((lookup))



#pedtry.loc[:, columns] = pedtry.loc[:, columns].isin(set(listaSnipow))

In [35]:
pedtry.head(5)

,FID,IID,Gender,AAlog_BMI,AGE,rs12620338,rs7559271,rs2234675,rs6436302,rs12053273,...,rs837550,rs2287072,rs112426189,rs3744374,rs12602590,rs11654604,rs200805689,rs117651561,rs79742527,rs143040759
0,3,1,2,1.410102,64.0,False,True,False,False,False,...,False,False,True,True,False,False,False,False,False,False
1,4,2,2,1.391112,29.0,False,False,False,True,True,...,False,False,False,False,False,True,False,False,False,False
2,5,3,1,1.381837,64.0,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,6,4,2,1.290702,21.0,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,7,5,2,1.494572,38.0,True,False,True,True,True,...,True,False,True,True,False,False,False,False,False,False


In [36]:
pedtry = pedtry.applymap(lambda x:1 if x == True else x)
pedtry = pedtry.applymap(lambda x:0 if x == False else x)
pedtry = pedtry.where(pedtry.AAlog_BMI != -9).dropna()
pedtry['Gender'] = pedtry['Gender'] - 1
newColList = pedtry.columns.tolist()
newColList.sort()

In [37]:
newColList[:5]

['AAlog_BMI', 'AGE', 'FID', 'Gender', 'IID']

In [38]:
pedtry = pedtry[newColList]
pedtry = pedtry.drop(columns=['FID','IID'])
pedtry.head(5)

,AAlog_BMI,AGE,Gender,rs1004299,rs1004930,rs1008400,rs1033046,rs10521300,rs10521307,rs1075440,...,rs9934504,rs9934800,rs9935401,rs9936365,rs9936385,rs9939609,rs9939973,rs9940128,rs9941349,rs9972796
0,1.410102,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.391112,29.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,1.381837,64.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
3,1.290702,21.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.494572,38.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [39]:
pedtry.to_csv("finalFile.csv", sep = ';')

In [ ]:
import os
cwd = os.getcwd()

In [ ]:
cwd

# 5 rs'ów najabrdziej związanych z otyłością

Szukamy rs'ów o największej korelacji z BMI, a następnie badamy ich korelację z innymi rs'ami, tak by wiedzą o wystąpieniu rs'a skorelowane z rs'em odpowiadającym za zwiększone BMI móc oszacować szansę na otyłość

In [40]:
from pandas.plotting import scatter_matrix

In [51]:
corr_matrix = pedtry.corr(method='pearson')

In [64]:
corr_matrix['AAlog_BMI'].sort_values(ascending=False)[0:5]

AAlog_BMI      1.000000
AGE            0.340691
rs140436257    0.046075
rs143040759    0.041598
rs11640012     0.036893
Name: AAlog_BMI, dtype: float64

In [63]:
corr_matrix['rs143040759'].sort_values(ascending=False)[0:4]

rs143040759    1.000000
rs112426189    0.111221
rs3744374      0.090359
rs2234675      0.046253
Name: rs143040759, dtype: float64

## Wyjaśnienie  
BMI jest zkorelowane z rs143040759, a rs143040759 jest skorelowany z rs3744374. Oznacza to, że wystąpienie zmiany na rs3744374 zwiększa prawdopodobieństwo zmiany na rs143040759 z czym wiąże się większe BMI

In [65]:
corr_matrix['rs140436257'].sort_values(ascending=False)[0:4]

rs140436257    1.000000
AAlog_BMI      0.046075
rs7589708      0.036517
rs2588996      0.031155
Name: rs140436257, dtype: float64

In [66]:
corr_matrix['rs143040759'].sort_values(ascending=False)[0:4]

rs143040759    1.000000
rs112426189    0.111221
rs3744374      0.090359
rs2234675      0.046253
Name: rs143040759, dtype: float64

In [68]:
corr_matrix['rs11640012'].sort_values(ascending=False)[0:4]

rs11640012    1.000000
rs12447674    0.220871
rs12931301    0.192724
rs1362437     0.074955
Name: rs11640012, dtype: float64

In [69]:
corr_matrix['rs143040759'].sort_values(ascending=False)[0:4]

rs143040759    1.000000
rs112426189    0.111221
rs3744374      0.090359
rs2234675      0.046253
Name: rs143040759, dtype: float64

In [72]:
import re


In [73]:
string = "test*"

In [80]:
znalezione = string.split('*')

In [81]:
znalezione

['test', '']